#掛載雲端

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 載入 AOI瑕疵檢測 資料集

In [ ]:
#將壓縮檔複製到/content
! cp  "/content/drive/MyDrive/train_images.zip" /content/

In [ ]:
#解壓縮訓練集
! unzip /content/train_images > data_unzip.log

In [ ]:
#read csv
import pandas as pd
AOI_data = pd.read_csv('/content/drive/MyDrive/AIdea_AOI/train.csv')

In [ ]:
#查看訓練集每類別圖片數量並且分類
label = []
for i in range(6):
  #請輸入程式碼


第0類張數: 674
第1類張數: 492
第2類張數: 100
第3類張數: 378
第4類張數: 240
第5類張數: 644


In [ ]:
import cv2
train_images = []
train_label = []
test_images = []
test_label = []
for i in range(6):
  #讀取圖片測試集圖片(這邊的程式碼不可更動)
  images_temp = []
  label_temp = [i] * 20
  for j in range(20):
    img = cv2.imread('/content/train_images/'+label[i]['ID'][j])
    images_temp.append(cv2.resize(img,(224,224), cv2.INTER_AREA))
  test_images += images_temp
  test_label += label_temp

  #將剩餘圖片進行增量，使訓練集每類張數相同
  #請輸入程式碼

  train_images += images_temp
  train_label += label_temp


In [ ]:
#將 list 轉成 array
import numpy as np
from sklearn.utils import shuffle
x_train = np.array(train_images)
x_test = np.array(test_images)
y_train = np.array(train_label)
y_test = np.array(test_label)

In [ ]:
#將將訓練集進行shuffle
import random
#請輸入程式碼


# 查看資料的 shape

In [ ]:
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)

x_train shape:  (1920, 224, 224, 3)
x_test shape:  (120, 224, 224, 3)


# 查看標籤的 shape

In [ ]:
print('y_train shape:',y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (1920,)
y_test shape: (120,)


# 查看標籤的內容 

In [ ]:
print(y_train[0:10])

[4 0 0 2 0 3 5 2 3 2]


# 進行 min-max normalization

In [ ]:
x_train_norm = x_train.astype('float32')/255
x_test_norm = x_test.astype('float32')/255

In [ ]:
print(x_train_norm[0][0][0])

[0.6156863 0.6156863 0.6156863]


# 將數字標籤進行 One-hot 編碼

In [ ]:
from tensorflow.keras import utils

# 轉換前
print(y_train[0])

1


In [ ]:
# 進行 One-hot 編碼轉換...
y_train_onehot = utils.to_categorical(y_train, 6)
y_test_onehot = utils.to_categorical(y_test, 6)
# 轉換後
print(y_train_onehot[0])

[0. 1. 0. 0. 0. 0.]


# 建立 CNN 神經網路架構

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import tensorflow.keras.applications as tensorflow_model

In [ ]:
vgg16 = tensorflow_model.VGG16() #請輸入參數

In [ ]:
#修改全聯階層的輸出類別數
#請輸入程式碼

# Create your own model 
cnn = keras.models.Model(inputs = vgg16.input, outputs=x) 
cnn.summary()

In [ ]:
# 神經網路的訓練配置 #
adam = tf.optimizers.Adam(0.0001)
cnn.compile(optimizer = adam,loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
# 進行訓練 #
history = cnn.fit(x=x_train_norm,y=y_train_onehot,batch_size=32,epochs=20,validation_split=0.1)

# 使用測試資料評估神經網路

In [ ]:
# evaluate
test_loss,test_val = cnn.evaluate(x_test_norm,y_test_onehot)
print('測試資料損失值:', test_loss)
print('測試資料準確度:', test_val)

4/4 [==============================] - 0s 34ms/step - loss: 0.3151 - acc: 0.9000
測試資料損失值: 0.3150562047958374
測試資料準確度: 0.8999999761581421


In [ ]:
# predict
predict_prop = cnn.predict(x_test_norm)
print('第一筆測試資料的預測機率', predict_prop[0])

第一筆測試資料的預測機率 [5.3620076e-01 9.1038318e-03 3.1686342e-01 1.3646436e-01 1.3534890e-03
 1.4220403e-05]


# 將模型轉換成ONNX

In [ ]:
#安裝套件 tf2onnx
!pip install tf-estimator-nightly==2.8.0.dev2021122109 #colab環境缺少此套件 需安裝此套件才能完整安裝tf2onnx
!pip install git+https://github.com/onnx/tensorflow-onnx

In [ ]:
#進行轉換
import tf2onnx
#請輸入程式碼


In [ ]:
#檢查opset版本
#請輸入程式碼


In [ ]:
#由於耐能的量化過程只支持opset ai.onnx 所以移除ai.onnx.ml並儲存
#請輸入程式碼
